# FALKOR | Automated Trading Platform

### Training PyTorch models for stock price prediction

GitHub Link: https://github.com/vdyagilev/FALKOR

In [1]:
from api_wrappers.BinanceWrapper import BinanceWrapper
from helpers import data_processing
from helpers.charting_tools import Charting
from helpers.datasets import ChartImageDataset, ArrayTimeSeriesDataset
from helpers.saving_models import load_model, save_model

from models.CNN.CNN import CNN
from models.GRU.GRU import GRUnet
from models.GRU_CNN.GRU_CNN import GRU_CNN

from pathlib import Path
import multiprocessing
import numpy as np
from tqdm import tqdm_notebook as tqdm

import torch
from torch.utils.data import *
import torchvision

In [2]:
import warnings
warnings.filterwarnings("ignore")

#### Import Data

In [3]:
id_ = 'nBjgb83VMNvqq45b3JdWUIsJDalWlXxHI2bvDz9oLdW7KgOLPvJCp30CHnthjfNJ'
sec = '5bBN7s7h37kUvmGIpF9FTAtspBY93WirwhTh39PV7AlKSlUE2S4EEe9b3OZVYIqd'

binance = BinanceWrapper(id_, sec)

symbol = 'ETHBTC'
interval = '1m'

def p(d1, d2):
    return binance.historical_candles(symbol, interval, d1, d2)
ochls = [p('April 1 2018', 'May 15 2018'), p('May 15 2018', 'July 1 2018'), p('July 1 2018', 'August 15 2018'), p('August 15 2018', 'October 1 2018'),
        p('October 1 2018', 'November 15 2018'), p('November 15 2018', 'January 1 2019'), p('January 1 2019', 'February 15 2019'), p('February 15 2019', 'April 1 2019'),
        p('April 1 2019', 'May 15 2019'), p('May 15 2019', 'July 1 2019'), p('July 1 2019', 'August 15 2019')]

#ochlv_df = binance.historical_candles(symbol, interval, 'April 1 2018', 'May 15 2018')

In [4]:
def train_cnn(ochlv_df):
    ti_df = data_processing.add_ti(ochlv_df)

    # return = (future - curr) / curr

    price_return = ((ti_df.close)-(ti_df.close.shift(1))) / ti_df.close
    volume_return = ((ti_df.volume)-(ti_df.volume.shift(1))) / ti_df.volume

    ti_df['price_return'] = price_return
    ti_df['volume_return'] = volume_return

    dataset_windows = data_processing.split_dataset(ti_df, a=0, b=30, step_size=5)
    for df in dataset_windows: df.reset_index(drop=True, inplace=True) # reindex 0-29

    # Paths to training images and testing images
    train_img_path = Path('models/CNN/training-images/')
    image_path_list = [train_img_path / 'image-{}.png'.format(i) for i in range(len(dataset_windows))]

    p = multiprocessing.Pool(processes = 6)
    p.map_async(generate_chart_image, [i for i in range(len(dataset_windows))])

    p.close()
    p.join()

    price_labels_dct = data_processing.price_labels(dataset_windows, period_size=30)
    price_returns = price_labels_dct['return']

    # ensure len(dataset_windows) == len(price_returns)
    dataset_windows = dataset_windows[:len(price_returns)]

    for i in range(len(dataset_windows)): normalize_data(i, dataset_windows)

    # This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware
    torch.backends.cudnn.benchmark = True

    # Parameters
    params = {'batch_size': 64,
              'shuffle': True,
              'num_workers': 5}

    num_epochs = 5

    # Remove all NaN values - in our case only the first window has a NaN so we remove first window and label
    dataset_windows = dataset_windows[1:]
    price_returns = price_returns[1:] 
    image_path_list = image_path_list[1:]
    curr_prices = price_labels_dct['curr_price'][1:]
    future_prices = price_labels_dct['future_price'][1:]

    while len(dataset_windows) % params['batch_size'] != 0:
        dataset_windows = dataset_windows[:-1]

    while len(curr_prices) % params['batch_size'] != 0:
        curr_prices = curr_prices[:-1]

    while len(future_prices) % params['batch_size'] != 0:
        future_prices = future_prices[:-1]

    while len(price_returns) % params['batch_size'] != 0:
        price_returns = price_returns[:-1]

    image_path_list = image_path_list[:len(price_returns)]

    assert len(dataset_windows) == len(price_returns) == len(curr_prices) == len(future_prices) == len(image_path_list)

    # specify the split between train_df and valid_df from the process of splitting dataset_windows 
    split = 0.7

    s = int(len(dataset_windows) * 0.7)
    while s % params['batch_size'] != 0:
        s += 1

    # create two ChartImageDatasets, split by split, for the purpose of creating a DataLoader for the specific model

    train_ds_cnn = ChartImageDataset(image_path_list[:s], price_returns[:s])
    valid_ds_cnn = ChartImageDataset(image_path_list[s:], price_returns[s:])

    # add potential profit as label
    test_ds_cnn = ChartImageDataset(image_path_list[s:], [future_prices[i] - curr_prices[i] for i in range(s, len(future_prices))])

    train_gen_cnn = DataLoader(train_ds_cnn, **params)
    valid_gen_cnn = DataLoader(valid_ds_cnn, **params)
    train_gen_cnn = DataLoader(valid_ds_cnn, **params)

    cnn = CNN().cuda()

    cnn_path = Path('models/CNN/cnn_weights')
    load_model(cnn, cnn_path)

    train(cnn, num_epochs, batch_size=params['batch_size'], train_gen=train_gen_cnn, valid_gen=valid_gen_cnn, test_gen=train_gen_cnn)

    save_model(cnn, cnn_path)

In [11]:
for o in ochls:
    train_cnn(o)

Epoch: 1/5... Training Loss: 0.0034 Validation Loss: 0.0021 Profitability: 4.007
Epoch: 2/5... Training Loss: 0.0016 Validation Loss: 0.0011 Profitability: 4.685
Epoch: 3/5... Training Loss: 0.0014 Validation Loss: 0.0011 Profitability: 4.68
Epoch: 4/5... Training Loss: 0.0012 Validation Loss: 0.0014 Profitability: 4.507
Epoch: 5/5... Training Loss: 0.001 Validation Loss: 0.0013 Profitability: 4.567
Epoch: 1/5... Training Loss: 0.0023 Validation Loss: 0.0024 Profitability: 2.11
Epoch: 2/5... Training Loss: 0.0016 Validation Loss: 0.0023 Profitability: 2.09
Epoch: 3/5... Training Loss: 0.0015 Validation Loss: 0.0016 Profitability: 2.775
Epoch: 4/5... Training Loss: 0.0014 Validation Loss: 0.0015 Profitability: 3.04
Epoch: 5/5... Training Loss: 0.0015 Validation Loss: 0.0014 Profitability: 3.031
Epoch: 1/5... Training Loss: 0.0027 Validation Loss: 0.0022 Profitability: 3.58
Epoch: 2/5... Training Loss: 0.0015 Validation Loss: 0.0013 Profitability: 4.186
Epoch: 3/5... Training Loss: 0.001

#### Add Technical Indicators

In [ ]:
ti_df = data_processing.add_ti(ochlv_df)

In [ ]:
ti_df.shape

#### Add price returns and volume returns as columns

In [ ]:
# return = (future - curr) / curr

price_return = ((ti_df.close)-(ti_df.close.shift(1))) / ti_df.close
volume_return = ((ti_df.volume)-(ti_df.volume.shift(1))) / ti_df.volume

ti_df['price_return'] = price_return
ti_df['volume_return'] = volume_return

In [ ]:
dataset_windows = data_processing.split_dataset(ti_df, a=0, b=30, step_size=5)
for df in dataset_windows: df.reset_index(drop=True, inplace=True) # reindex 0-29
print(len(dataset_windows))

#### Generate Chart Images for CNN

In [ ]:
# Paths to training images and testing images
train_img_path = Path('models/CNN/training-images/')
image_path_list = [train_img_path / 'image-{}.png'.format(i) for i in range(len(dataset_windows))]

In [7]:
def generate_chart_image(i):
    df = dataset_windows[i]
    chart = Charting(df=df, col_label='time', row_label='close', tech_inds=['sma20', 'bb20_low', 'bb20_mid', 'bb20_up'])
    chart.chart_to_image(train_img_path / 'image-{}.png'.format(i)) # the / is a Path join method 
    
    del chart

# p = multiprocessing.Pool(processes = 6)
# p.map_async(generate_chart_image, [i for i in range(len(dataset_windows))])

# p.close()
# p.join()

#### Normalizing input DataFrames

In [ ]:
price_labels_dct = data_processing.price_labels(dataset_windows, period_size=30)
price_returns = price_labels_dct['return']

# ensure len(dataset_windows) == len(price_returns)
dataset_windows = dataset_windows[:len(price_returns)]
len(dataset_windows)

In [10]:
def normalize_data(i, dataset_windows):
    df = dataset_windows[i]
    
    # remove all hardcoded features like time open high low close, which have 0 predictive ability
    if 'time' in df: df = df.drop('time', axis=1)
#     if 'open' in df: df = df.drop('open', axis=1)
#     if 'high' in df: df = df.drop('high', axis=1)
#     if 'low' in df: df = df.drop('low', axis=1)
#     if 'close' in df: df = df.drop('close', axis=1)
#     if 'volume' in df: df = df.drop('volume', axis=1)
        
    df = df.fillna(0) # replace all NaN with 0. 
   
    dataset_windows[i] = df
    
# for i in range(len(dataset_windows)): normalize_data(i, dataset_windows)
# dataset_windows[0].head()

### Training

In [ ]:
# This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware
torch.backends.cudnn.benchmark = True

# Parameters
params = {'batch_size': 64,
          'shuffle': False,
          'num_workers': 5}

num_epochs = 20

In [ ]:
# Remove all NaN values - in our case only the first window has a NaN so we remove first window and label
dataset_windows = dataset_windows[1:]
price_returns = price_returns[1:] 
image_path_list = image_path_list[1:]
curr_prices = price_labels_dct['curr_price'][1:]
future_prices = price_labels_dct['future_price'][1:]

In [ ]:
while len(dataset_windows) % params['batch_size'] != 0:
    dataset_windows = dataset_windows[:-1]
    
while len(curr_prices) % params['batch_size'] != 0:
    curr_prices = curr_prices[:-1]

while len(future_prices) % params['batch_size'] != 0:
    future_prices = future_prices[:-1]

while len(price_returns) % params['batch_size'] != 0:
    price_returns = price_returns[:-1]

image_path_list = image_path_list[:len(price_returns)]

assert len(dataset_windows) == len(price_returns) == len(curr_prices) == len(future_prices) == len(image_path_list)

In [8]:
def train(model, num_epochs, batch_size, train_gen, valid_gen, test_gen, gru=False):
    """Standard training function used by all three models""" 
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    # loop through the dataset num_epoch times
    for epoch in range(num_epochs):
               
        # train loop
        train_loss = []
        valid_loss = []
        
        # take the batch and labels for batch 
        for batch, labels in train_gen:     
            if gru:
                # add extra dimension to every vector in batch
                batch.unsqueeze_(-1)
                batch = batch.expand(batch.shape[0], batch.shape[1], 1)
                
                # reformat dimensions
                batch = batch.transpose(2,0)
                batch = batch.transpose(1, 2)
                
            batch, labels = batch.cuda(), labels.cuda()
            batch, labels = batch.float(), labels.float()
            
            # clear gradients
            model.zero_grad()
            output = model(batch)
            
            # change the output from whatever matrix to a vector
            if gru:
                output = output[0]
                
            output = output.flatten()
            
            # we use the RMSE error function to train our model
            criterion = torch.nn.MSELoss()
            
            loss = torch.sqrt(criterion(output, labels))
            
            # backpropogate loss through model
            loss.backward()

            # perform model training based on propogated loss
            optimizer.step()
            
            train_loss.append(loss)
            
        # validation loop
        
        profit = 0
        with torch.set_grad_enabled(False):
            for batch, labels in valid_gen:
                if gru:
                    # add extra dimension to every vector in batch
                    batch.unsqueeze_(-1)
                    batch = batch.expand(batch.shape[0], batch.shape[1], 1)

                    # reformat dimensions
                    batch = batch.transpose(2,0)
                    batch = batch.transpose(1, 2)
                    
                batch, labels = batch.cuda(), labels.cuda()
                batch, labels = batch.float(), labels.float()
                
                # transform the model from training configuration to testing configuration. ex. dropout layers are removed
                model.eval()

                output = model(batch)
                
                if gru:
                    output = output[0] # turn (1, batch_size, 1) to (batch_size, 1)
                
                output = output.flatten() # turn (batch_size, 1) to (batch_size)
                
                val_loss = torch.sqrt(criterion(output, labels))
                
                model.train()
                
                valid_loss.append(val_loss)
                
            
            # Profitability testing
            profit = 0.0
            
            for batch, labels in test_gen:
                if gru:
                    # add extra dimension to every vector in batch
                    batch.unsqueeze_(-1)
                    batch = batch.expand(batch.shape[0], batch.shape[1], 1)

                    # reformat dimensions
                    batch = batch.transpose(2,0)
                    batch = batch.transpose(1, 2)
                
                batch, labels = batch.cuda(), labels.cuda()
                batch, labels = batch.float(), labels.float()
                
                # transform the model from training configuration to testing configuration. ex. dropout layers are removed
                model.eval()
                
                output = model(batch)
                
                if gru:
                    output = output[0] # turn (1, batch_size, 1) to (batch_size, 1)
                
                output = output.flatten() # turn (batch_size, 1) to (batch_size)
                
                # if output is > 0 ==> model predict positive growth for the next five cycles. Purchase now and sell in 5 periods.
                for i, pred in enumerate(output):
                    if pred > 0: # price will increase
                        profit += labels[i]
                       
                model.train()
                
                
        print("Epoch: {}/{}...".format(epoch+1, num_epochs),
              "Training Loss: {}".format(round(float(sum(train_loss)/len(train_loss)), 4)),
              "Validation Loss: {}".format(round(float(sum(valid_loss)/len(valid_loss)), 4)),
              "Profitability: {}".format(round(float(profit), 3)))     

In [9]:
def train_dual(model, num_epochs, batch_size, train_gen1, train_gen2, valid_gen1, valid_gen2, test_gen, gru=False):
    """Standard training function used by all three models"""
    # For optimizing our model, we choose SGD 
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
    
    # training loop
    
    # toop through the dataset num_epoch times
    for epoch in range(num_epochs):
        
        # train loop
        
        train_loss = []
        valid_loss = []
        
        # loop through each batch
        for i  in range(batch_size):
            gru_batch, gru_labels = next(iter(train_gen1))
            gru_batch, gru_labels = gru_batch.cuda(), gru_labels.cuda()
            gru_batch, gru_labels = gru_batch.float(), gru_labels.float()
            
            # add extra dimension to every vector in batch
            gru_batch.unsqueeze_(-1)
            gru_batch = gru_batch.expand(gru_batch.shape[0], gru_batch.shape[1], 1)

            # reformat dimensions
            gru_batch = gru_batch.transpose(2,0)
            gru_batch = gru_batch.transpose(1, 2)
            cnn_batch, cnn_labels = next(iter(train_gen2))
            cnn_batch, cnn_labels = cnn_batch.cuda(), cnn_labels.cuda()
            cnn_batch, cnn_labels = cnn_batch.float(), cnn_labels.float()
            
            # clear gradients
            model.zero_grad()
            output = model(gru_batch, cnn_batch)
            output = output[0]
            # declare the loss function and calculate output loss
            
            # we use the RMSE error function to train our model
            criterion = torch.nn.MSELoss()
            
            loss = torch.sqrt(criterion(output, gru_labels))
            
            # backpropogate loss through model
            loss.backward()
            # perform model training based on propogated loss
            optimizer.step()
            
            train_loss.append(loss)
        
        
        # validation loop
        with torch.set_grad_enabled(False):
            for i in range(batch_size):
                gru_batch, gru_labels = next(iter(valid_gen1))
                gru_batch, gru_labels = gru_batch.cuda(), gru_labels.cuda()
                gru_batch, gru_labels = gru_batch.float(), gru_labels.float()
                
                # add extra dimension to every vector in batch
                gru_batch.unsqueeze_(-1)
                gru_batch = gru_batch.expand(gru_batch.shape[0], gru_batch.shape[1], 1)

                # reformat dimensions
                gru_batch = gru_batch.transpose(2,0)
                gru_batch = gru_batch.transpose(1, 2)

                cnn_batch, cnn_labels = next(iter(valid_gen2))
                cnn_batch, cnn_labels = cnn_batch.cuda(), cnn_labels.cuda()
                cnn_batch, cnn_labels = cnn_batch.float(), cnn_labels.float()
                
                # transform the model from training configuration to testing configuration. ex. dropout layers are removed
                model.eval()

                output = model(gru_batch, cnn_batch)
                output = output[0]
                
                val_loss = torch.sqrt(criterion(output, gru_labels))
                
                model.train()
                
                valid_loss.append(val_loss)
                
             # Profitability testing
            profit = 0.0
            
            for batch, labels in test_gen:
                gru_batch, gru_labels = next(iter(valid_gen1))
                gru_batch, gru_labels = gru_batch.cuda(), gru_labels.cuda()
                gru_batch, gru_labels = gru_batch.float(), gru_labels.float()
                
                # add extra dimension to every vector in batch
                gru_batch.unsqueeze_(-1)
                gru_batch = gru_batch.expand(gru_batch.shape[0], gru_batch.shape[1], 1)

                # reformat dimensions
                gru_batch = gru_batch.transpose(2,0)
                gru_batch = gru_batch.transpose(1, 2)

                cnn_batch, cnn_labels = next(iter(valid_gen2))
                cnn_batch, cnn_labels = cnn_batch.cuda(), cnn_labels.cuda()
                cnn_batch, cnn_labels = cnn_batch.float(), cnn_labels.float()
                
                # transform the model from training configuration to testing configuration. ex. dropout layers are removed
                model.eval()

                output = model(gru_batch, cnn_batch)
                output = output[0]
                
                # if output is > 0 ==> model predict positive growth for the next five cycles. Purchase now and sell in 5 periods.
                
                for i, pred in enumerate(output):
                    if pred > 0: # price will increase
                        profit += labels[i]
                
                
                model.train()
                
        print("Epoch: {}/{}...".format(epoch+1, num_epochs),
              "Training Loss: {}".format(round(float(sum(train_loss)/len(train_loss)), 4)),
              "Validation Loss: {}".format(round(float(sum(valid_loss)/len(valid_loss)), 4)),
              "Profitability: {}".format(round(float(profit), 3)))            

In [ ]:
# specify the split between train_df and valid_df from the process of splitting dataset_windows 
split = 0.7

s = int(len(dataset_windows) * 0.7)
while s % params['batch_size'] != 0:
    s += 1

### Training CNN

In [ ]:
# create two ChartImageDatasets, split by split, for the purpose of creating a DataLoader for the specific model

train_ds_cnn = ChartImageDataset(image_path_list[:s], price_returns[:s])
valid_ds_cnn = ChartImageDataset(image_path_list[s:], price_returns[s:])

# add potential profit as label
test_ds_cnn = ChartImageDataset(image_path_list[s:], [future_prices[i] - curr_prices[i] for i in range(s, len(future_prices))])

In [ ]:
train_gen_cnn = DataLoader(train_ds_cnn, **params)
valid_gen_cnn = DataLoader(valid_ds_cnn, **params)
train_gen_cnn = DataLoader(valid_ds_cnn, **params)

In [ ]:
cnn = CNN().cuda()

In [ ]:
cnn_path = Path('models/CNN/cnn_weights')
load_model(cnn, cnn_path)

In [ ]:
train(cnn, num_epochs, batch_size=params['batch_size'], train_gen=train_gen_cnn, valid_gen=valid_gen_cnn, test_gen=train_gen_cnn)

In [ ]:
save_model(cnn, cnn_path)

### Training GRU

In [ ]:
train_ds_gru = ArrayTimeSeriesDataset(dataset_windows[:s], price_returns[:s])
valid_ds_gru = ArrayTimeSeriesDataset(dataset_windows[s:], price_returns[s:])
test_ds_gru = ArrayTimeSeriesDataset(dataset_windows[s:], 
                                     [future_prices[i] - curr_prices[i] for i in range(s, len(future_prices))])

In [ ]:
hidden_size = 800

train_gen_gru = DataLoader(train_ds_gru, **params)
valid_gen_gru = DataLoader(valid_ds_gru, **params)
test_gen_gru = DataLoader(valid_ds_gru, **params)

In [ ]:
gru = GRUnet(num_features=390, batch_size=params['batch_size'], hidden_size=hidden_size).float().cuda()

In [ ]:
gru_path = Path('models/GRU/gru_weights')
load_model(gru, gru_path)

In [ ]:
train(gru, num_epochs, batch_size=params['batch_size'], train_gen=train_gen_gru, valid_gen=valid_gen_gru, test_gen=test_gen_gru, gru=True)

In [ ]:
save_model(gru, gru_path)

### Training GRU-CNN

In [ ]:
gru_cnn = GRU_CNN(num_features=390, batch_size=params['batch_size'], hidden_size=800).float().cuda()

In [ ]:
gru_cnn_path = Path('models/CNN_GRU/gcnn_gru_weights')
load_model(gru_cnn, gru_cnn_path)

In [ ]:
gru_cnn.load_cnn_weights(cnn)
gru_cnn.load_gru_weights(gru)

In [ ]:
train_dual(gru_cnn, num_epochs, batch_size=params['batch_size'], train_gen1=train_gen_gru, train_gen2=train_gen_cnn,
           valid_gen1=valid_gen_gru, valid_gen2=valid_gen_cnn, test_gen=test_gen_gru, gru=True)

In [ ]:
save_model(gru_cnn, gru_cnn_path)